<a href="https://colab.research.google.com/github/Aisyaululasmi/Assignment/blob/main/CVL_Assignment_2_Aisya_Ulul_Asmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2 - Computer Vision Lanjut

Implement one of the object detection methods to solve certain problem without using library. Give analysis of your results in term of accuracy and IoU.

### Nama : Aisya Ulul Asmi
### NIM : 25/564969/PPA/07123

In [ ]:
%pip install Pillow

###Konversi JPG ke format ASCII RGB

Digunakan library pillow untuk melakukan konversi jpg ke ASCII

In [62]:
from PIL import Image
import os
import sys

#Mengkonversi gambar JPG ke format ASCII RGB
def convert_jpg_to_ascii_rgb(jpg_path, output_dir="sample_images", ascii_filename=None, target_width=None, target_height=None):
    try:
        img = Image.open(jpg_path).convert('RGB') #Cek gambar dalam mode RGB

        if target_width and target_height:
            img = img.resize((target_width, target_height))
        elif target_width: #Resize jika hanya width yang ditentukan
            aspect_ratio = img.height / img.width
            new_height = int(target_width * aspect_ratio)
            img = img.resize((target_width, new_height))
        elif target_height: #Resize jika hanya height yang ditentukan
            aspect_ratio = img.width / img.height
            new_width = int(target_height * aspect_ratio)
            img = img.resize((new_width, target_height))

        width, height = img.size

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        if ascii_filename is None:
            base_name = os.path.splitext(os.path.basename(jpg_path))[0]
            ascii_filename = f"{base_name}.txt"

        output_path = os.path.join(output_dir, ascii_filename)

        with open(output_path, 'w') as f:
            f.write(f"{width},{height}\n") #Dimensi

            for y in range(height):
                row_pixels = []
                for x in range(width):
                    r, g, b = img.getpixel((x, y))
                    row_pixels.append(f"{r},{g},{b}")
                f.write(";".join(row_pixels) + "\n")

        print(f"Berhasil mengkonversi '{jpg_path}' ke '{output_path}' ({width}x{height})")
        return output_path

    except FileNotFoundError:
        print(f"File '{jpg_path}' tidak ditemukan.", file=sys.stderr)
        return None
    except Exception as e:
        print(f"Terjadi kesalahan saat mengkonversi '{jpg_path}': {e}", file=sys.stderr)
        return None

if __name__ == "__main__":
    #Membuat folder untuk gambar JPG input
    input_jpg_dir = "input_jpg_images"
    os.makedirs(input_jpg_dir, exist_ok=True)

    #Folder output untuk file ASCII
    output_ascii_dir = "sample_images"
    os.makedirs(output_ascii_dir, exist_ok=True)

    #Daftar gambar JPG yang akan dikonversi
    jpg_files_to_convert = [
        "apple.jpg",
        "orange.jpg",
        "cherry.jpg"
    ]

    converted_ascii_paths = []
    for jpg_file in jpg_files_to_convert:
        jpg_full_path = os.path.join(input_jpg_dir, jpg_file)

        #Konversi dan simpan ke folder sample_images
        ascii_path = convert_jpg_to_ascii_rgb(jpg_full_path,
                                              output_dir=output_ascii_dir,
                                              target_width=50,
                                              target_height=50)
        if ascii_path:
            converted_ascii_paths.append(ascii_path)

if converted_ascii_paths:
  print("\nKonversi JPG ke ASCII selesai.")
else:
  print("\nTidak ada gambar JPG yang berhasil dikonversi.")

Berhasil mengkonversi 'input_jpg_images/apple.jpg' ke 'sample_images/apple.txt' (50x50)
Berhasil mengkonversi 'input_jpg_images/orange.jpg' ke 'sample_images/orange.txt' (50x50)
Berhasil mengkonversi 'input_jpg_images/cherry.jpg' ke 'sample_images/cherry.txt' (50x50)

Konversi JPG ke ASCII selesai.


###Program Deteksi Buah Sederhana

In [69]:
import os
import sys

class SimpleColorDetector:
    def __init__(self):
        self.results_dir = "results"
        self.visualization_ascii_dir = os.path.join(self.results_dir, "visualization_ascii")
        os.makedirs(self.visualization_ascii_dir, exist_ok=True)
        self.detection_results_file = os.path.join(self.results_dir, "detection_results.txt")

    #Membaca gambar ASCII dan konversi ke matriks RGB.
    def _load_image_from_ascii(self, image_path):
        try:
            with open(image_path, 'r') as f:
                lines = f.readlines()

            if not lines:
                raise ValueError("File gambar ASCII kosong.")

            dim_line = lines[0].strip().split(',')
            if len(dim_line) != 2:
                raise ValueError("Format dimensi gambar ASCII tidak valid.")
            width, height = int(dim_line[0]), int(dim_line[1])

            image_data = []
            for line_idx, line in enumerate(lines[1:]): #Lewati baris dimensi
                pixels_str = line.strip().split(';')
                row_data = []
                for pixel_str in pixels_str:
                    if not pixel_str: continue
                    rgb_values = pixel_str.split(',')
                    if len(rgb_values) == 3:
                        try:
                            r, g, b = int(rgb_values[0]), int(rgb_values[1]), int(rgb_values[2])
                            row_data.append((r, g, b))
                        except ValueError:
                            print(f"Warning: Invalid RGB value in line {line_idx+2}: '{pixel_str}'. Skipping.", file=sys.stderr)
                            row_data.append((0, 0, 0)) #Default hitam
                    else:
                        print(f"Warning: Invalid pixel format in line {line_idx+2}: '{pixel_str}'. Skipping.", file=sys.stderr)
                        row_data.append((0, 0, 0)) # Default hitam
                if row_data:
                    image_data.append(row_data)

            #Verifikasi dimensi
            if len(image_data) != height or (image_data and len(image_data[0]) != width):
                print(f"Dimensi gambar ASCII tidak sesuai dengan deklarasi. Deklarasi : {width}x{height}, Aktual : {len(image_data[0]) if image_data else 0}x{len(image_data)}. Menyesuaikan.", file=sys.stderr)
                #Sesuaikan width/height jika dimensi gambar berbeda
                height = len(image_data)
                width = len(image_data[0]) if image_data else 0

            return image_data, width, height
        except Exception as e:
            print(f"Error mengambil gambar dari ASCII '{image_path}': {e}", file=sys.stderr)
            return None, 0, 0

    #Menghitung jarak Euclidean antara dua titik warna (R, G, B)
    def _euclidean_distance(self, p1, p2):
        return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

    #Membuat masker biner berdasarkan jarak Euclidean warna
    def _create_binary_mask(self, image_data, target_color, threshold):
        if not image_data:
            return [], 0, 0

        height = len(image_data)
        width = len(image_data[0])

        binary_mask = [[0 for _ in range(width)] for _ in range(height)]
        for y in range(height):
            for x in range(width):
                pixel_color = image_data[y][x]
                if self._euclidean_distance(pixel_color, target_color) <= threshold:
                    binary_mask[y][x] = 1
        return binary_mask, width, height

    #Mendeteksi bounding box dari area yang terdeteksi
    def _detect_bounding_box(self, binary_mask):
        if not binary_mask:
            return None

        min_x, min_y = float('inf'), float('inf')
        max_x, max_y = float('-inf'), float('-inf')
        found_pixel = False

        height = len(binary_mask)
        width = len(binary_mask[0])

        for y in range(height):
            for x in range(width):
                if binary_mask[y][x] == 1:
                    found_pixel = True
                    min_x = min(min_x, x)
                    min_y = min(min_y, y)
                    max_x = max(max_x, x)
                    max_y = max(max_y, y)

        if found_pixel:
            return (min_x, min_y, max_x, max_y) # (x_min, y_min, x_max, y_max)
        else:
            return None

    #Menghitung intersection over union (IoU) antara dua bounding box
    def _calculate_iou(self, box1, box2):
        if box1 is None or box2 is None:
            return 0.0

        x_left = max(box1[0], box2[0])
        y_top = max(box1[1], box2[1])
        x_right = min(box1[2], box2[2])
        y_bottom = min(box1[3], box2[3])

        if x_right < x_left or y_bottom < y_top:
            return 0.0

        intersection_area = (x_right - x_left) * (y_bottom - y_top)

        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

        union_area = float(box1_area + box2_area - intersection_area)

        if union_area == 0:
            return 0.0

        return intersection_area / union_area

    #Membuat representasi ASCII dari bounding box
    def _generate_ascii_visualization(self, image_data, binary_mask, bounding_box, output_filename):
        if not image_data:
            return

        height = len(image_data)
        width = len(image_data[0])

        vis_lines = []
        for y in range(height):
            line = []
            for x in range(width):
                char = '.' #Default background

                if bounding_box and \
                   ((x == bounding_box[0] and y >= bounding_box[1] and y <= bounding_box[3]) or \
                    (x == bounding_box[2] and y >= bounding_box[1] and y <= bounding_box[3]) or \
                    (y == bounding_box[1] and x >= bounding_box[0] and x <= bounding_box[2]) or \
                    (y == bounding_box[3] and x >= bounding_box[0] and x <= bounding_box[2])):
                    char = '#' #Tepi bounding box
                elif binary_mask[y][x] == 1:
                    char = 'X' #Mendeteksi object pixel
                else:
                    r, g, b = image_data[y][x]
                    avg_intensity = (r + g + b) // 3
                    if avg_intensity > 200: char = ' '
                    elif avg_intensity > 150: char = '.'
                    elif avg_intensity > 100: char = '-'
                    else: char = '*'
                line.append(char)
            vis_lines.append("".join(line))

        output_path = os.path.join(self.visualization_ascii_dir, output_filename)
        with open(output_path, 'w') as f:
            f.write(f"Detected Object (X), Bounding Box (#), Background (*.- ):\n")
            f.write("\n".join(vis_lines))
        print(f"Visualisasi ASCII disimpan di folder : {output_path}")


    #Menjalankan deteksi objek untuk sebuah gambar
    def run_detection(self, image_path, target_color_rgb, color_threshold, ground_truth_box=None):
        print(f"\nMemproses gambar {image_path}")

        image_data, width, height = self._load_image_from_ascii(image_path)
        if image_data is None or not image_data:
            print(f"Tidak dapat memuat {image_path}.")
            return

        #1. Segmentasi berbasis warna
        print(f"Segmentasi berbasis warna {target_color_rgb} dengan threshold {color_threshold}")
        binary_mask, _, _ = self._create_binary_mask(image_data, target_color_rgb, color_threshold)

        #2. Mendeteksi Bounding Box
        print("Deteksi bounding box")
        detected_box = self._detect_bounding_box(binary_mask)

        #3. Perhitungan IoU
        iou = 0.0
        if ground_truth_box:
            iou = self._calculate_iou(detected_box, ground_truth_box)
            print(f"Ground Truth Box : {ground_truth_box}")
            print(f"Detected Box :     {detected_box}")
            print(f"IoU :              {iou:.4f}")
        else:
            print(f"Detected Box : {detected_box}")
            print("No ground truth box provided for IoU calculation.")

        #4. Accuracy
        accuracy_status = "Object Detected" if detected_box else "No Object Detected"
        print(f"Accuracy Status : {accuracy_status}")

        #Simpan hasil result di file
        with open(self.detection_results_file, 'a') as f:
            f.write(f"Image : {os.path.basename(image_path)}\n")
            f.write(f"  Target Color : {target_color_rgb}, Threshold : {color_threshold}\n")
            f.write(f"  Detected Box : {detected_box}\n")
            if ground_truth_box:
                f.write(f"  Ground Truth Box : {ground_truth_box}\n")
                f.write(f"  IoU : {iou:.4f}\n")
            f.write(f"  Accuracy Status : {accuracy_status}\n")
            f.write("-" * 30 + "\n")

        #Generate Visualisasi ASCII
        base_filename = os.path.splitext(os.path.basename(image_path))[0]
        self._generate_ascii_visualization(image_data, binary_mask, detected_box, f"{base_filename}_detection.txt")

if __name__ == "__main__":
    detector = SimpleColorDetector()

    #Hapus results sebelumnya
    if os.path.exists(detector.detection_results_file):
        os.remove(detector.detection_results_file)

    sample_images_dir = "sample_images"
    os.makedirs(sample_images_dir, exist_ok=True)


    #Mendefinisikan image paths, target colors, thresholds, dan ground truth boxes
    image_configs = [
        {
            "name": "apple",
            "path": os.path.join(sample_images_dir, "apple.txt"),
            "target_color": (180, 20, 20),  # Merah
            "threshold": 65,
            "ground_truth_box": (4, 12, 43, 45)
        },
        {
            "name": "orange",
            "path": os.path.join(sample_images_dir, "orange.txt"),
            "target_color": (255, 140, 0), # Oranye
            "threshold": 70,
            "ground_truth_box": (10, 7, 39, 40)
        },
        {
            "name": "cherry",
            "path": os.path.join(sample_images_dir, "cherry.txt"),
            "target_color": (170, 0, 30), # Merah tua (cherry)
            "threshold": 50,
            "ground_truth_box": (21, 27, 28, 38)
        }
    ]

    print("Menjalankan Deteksi Objek Buah-Buahan")
    for config in image_configs:
        if not os.path.exists(config["path"]):
            print(f"Error : Gambar '{config['path']}' tidak ditemukan. Mohon pastikan file ASCII gambar ada di folder 'sample_images/'.")
            continue
        detector.run_detection(
            image_path=config["path"],
            target_color_rgb=config["target_color"],
            color_threshold=config["threshold"],
            ground_truth_box=config["ground_truth_box"]
        )

    print("\nDeteksi objek selesai. Cek folder 'results/' untuk laporan dan visualisasi.")

Menjalankan Deteksi Objek Buah-Buahan

Memproses gambar sample_images/apple.txt
Segmentasi berbasis warna (180, 20, 20) dengan threshold 65
Deteksi bounding box
Ground Truth Box : (4, 12, 43, 45)
Detected Box :     (4, 12, 43, 45)
IoU :              1.0000
Accuracy Status : Object Detected
Visualisasi ASCII disimpan di folder : results/visualization_ascii/apple_detection.txt

Memproses gambar sample_images/orange.txt
Segmentasi berbasis warna (255, 140, 0) dengan threshold 70
Deteksi bounding box
Ground Truth Box : (10, 7, 39, 40)
Detected Box :     (10, 7, 39, 40)
IoU :              1.0000
Accuracy Status : Object Detected
Visualisasi ASCII disimpan di folder : results/visualization_ascii/orange_detection.txt

Memproses gambar sample_images/cherry.txt
Segmentasi berbasis warna (170, 0, 30) dengan threshold 50
Deteksi bounding box
Ground Truth Box : (21, 27, 28, 38)
Detected Box :     (21, 27, 28, 38)
IoU :              1.0000
Accuracy Status : Object Detected
Visualisasi ASCII disimpa

In [71]:
import os

results_file = "results/detection_results.txt"
visualization_dir = "results/visualization_ascii"

if os.path.exists(results_file):
    # Tampilkan visualisasi ASCII
    if os.path.exists(visualization_dir) and os.listdir(visualization_dir):
        print("Visualisasi ASCII")
        for filename in os.listdir(visualization_dir):
            if filename.endswith(".txt"):
                filepath = os.path.join(visualization_dir, filename)
                print(f"\nVisualisasi dari {filepath}:")
                try:
                    with open(filepath, 'r') as f:
                        print(f.read())
                except Exception as e:
                    print(f"Error membaca file visualisasi '{filepath}': {e}", file=sys.stderr)
        print("-------------------------")
    else:
        print(f"Folder visualisasi '{visualization_dir}' kosong atau tidak ditemukan.")

else:
    print(f"File hasil '{results_file}' tidak ditemukan.")

Visualisasi ASCII

Visualisasi dari results/visualization_ascii/apple_detection.txt:
Detected Object (X), Bounding Box (#), Background (*.- ):
                                                  
                               .-----..           
                             -**********.         
                           -************.         
                         .*************.          
                        .*************-           
                       .*************-            
                .      ****--*******.             
               **.     *----*****-.               
                .*.   .-.-------.                 
                 .*   . .....                     
                  .*..                            
    ########################################      
    #          .-XXX*XXXXXXX***--..-..     #      
    #     ..... .-*X**XXXXXX-----*****-.   #      
    #    --XXXX-----**----***XX*********-. #      
    #  .-XXXXXXXXXXX*XXXXX*****X*********

#Analisa

Berikut Ringkasan Hasil Deteksi
* Jumlah Total Gambar Diproses: 3
* Rata-rata IoU : 1.0000
* Tingkat Akurasi (berdasarkan status deteksi) : 100.00%

Analisis per Gambar:

1. apple.txt

Deteksi untuk gambar apel sangat akurat. Bounding box yang terdeteksi sama persis dengan ground truth, menghasilkan IoU sempurna (1.0). Visualisasi ASCII juga menunjukkan bounding box yang pas mengelilingi area objek yang terdeteksi.

2. orange.txt

Sama seperti apel, deteksi untuk gambar jeruk juga sangat akurat. IoU-nya 1.0, menandakan kesesuaian sempurna antara bounding box terdeteksi dan ground truth. Visualisasi ASCII mengkonfirmasi penempatan bounding box yang tepat.

3. cherry.txt

Deteksi untuk ceri juga menunjukkan hasil yang sempurna dengan IoU 1.0. Ini menunjukkan bahwa metode deteksi warna berhasil mengidentifikasi dan menempatkan bounding box dengan tepat sesuai ground truth.

###Kesimpulan Analisis:

Metode deteksi objek berbasis warna sederhana yang diimplementasikan menunjukkan kinerja yang sangat baik pada dataset gambar sampel yang kecil ini. Rata-rata IoU 1.0000 dan akurasi 100.00% menunjukkan bahwa metode ini berhasil mendeteksi dan menempatkan bounding box dengan sempurna untuk ketiga gambar sampel (apple.txt, orange.txt, dan cherry.txt) berdasarkan ground truth yang disediakan. Namun hal ini besar dikarenakan gambar sampel yang digunakan sangat sederhana, objeknya terisolasi dengan latar belakang yang kontras, dan ground truth bounding box yang disediakan sangat cocok dengan area yang terdeteksi oleh threshold warna.

Pada gambar yang lebih kompleks dengan latar belakang bervariasi, pencahayaan yang tidak merata, atau objek yang saling tumpang tindih, metode deteksi warna sederhana kemungkinan besar tidak akan memberikan hasil yang sesempurna ini.
